In [1]:
import numpy as np

In [2]:
X_train = np.load('Datasets/Fashion_MNIST/X_train.npy')
X_test = np.load('Datasets/Fashion_MNIST/X_test.npy')
y_train = np.load('Datasets/Fashion_MNIST/y_train.npy')
y_test = np.load('Datasets/Fashion_MNIST/y_test.npy')

In [3]:
X_train.shape

(60000, 784)

In [4]:
y_train.shape

(60000,)

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
encoder = OneHotEncoder(sparse=False)

In [7]:
encoder.fit(y_train.reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [8]:
y_train = encoder.transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

In [9]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

### Training

In [10]:
import tensorflow as tf

C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
tf.reset_default_graph()

In [12]:
Batch_size = 512
LR = 0.0005
epochs = 100

In [13]:
n_features = X_train.shape[1]
n_labels = y_train.shape[1]

In [14]:
X = tf.placeholder(tf.float32, shape=[None, n_features], name='features')
y = tf.placeholder(tf.float32, shape=[None, n_labels], name='labels')

In [15]:
Layers = [256, 128, 64, 32]

In [16]:
xavier = tf.contrib.layers.xavier_initializer()

In [17]:
W1 = tf.Variable(xavier([n_features, Layers[0]]))
b1 = tf.Variable(tf.zeros([Layers[0]]))

W2 = tf.Variable(xavier([Layers[0], Layers[1]]))
b2 = tf.Variable(tf.zeros([Layers[1]]))

W3 = tf.Variable(xavier([Layers[1], Layers[2]]))
b3 = tf.Variable(tf.zeros([Layers[2]]))

W4 = tf.Variable(xavier([Layers[2], Layers[3]]))
b4 = tf.Variable(tf.zeros([Layers[3]]))

W5 = tf.Variable(xavier([Layers[3], n_labels]))
b5 = tf.Variable(tf.zeros([n_labels]))

In [18]:
Z1 = tf.add(tf.matmul(X, W1), b1)
A1 = tf.tanh(Z1)

Z2 = tf.add(tf.matmul(A1, W2), b2)
A2 = tf.tanh(Z2)

Z3 = tf.add(tf.matmul(A2, W3), b3)
A3 = tf.tanh(Z3)

Z4 = tf.add(tf.matmul(A3, W4), b4)
A4 = tf.tanh(Z4)

Z5 = tf.add(tf.matmul(A4, W5), b5)

correct_pred = tf.equal(tf.argmax(Z5,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=Z5)

cost = tf.reduce_mean(loss)

train_op = tf.train.AdamOptimizer(learning_rate=LR).minimize(cost)

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(1, epochs+1):
        for i, (X_batch, y_batch) in enumerate(get_next_batch(X_train, y_train, Batch_size)):
            _, c, train_acc = sess.run([train_op, cost, accuracy], {X: X_batch, y: y_batch})
            
        if epoch%10 == 0:
            test_acc, = sess.run([accuracy], {X: X_test, y: y_test})
            print("Epoch:", epoch, " Loss:", c, " Train accuracy:", train_acc, " Test accuracy:", test_acc)

Epoch: 10  Loss: 0.5545477  Train accuracy: 0.7988281  Test accuracy: 0.7842
Epoch: 20  Loss: 0.5009248  Train accuracy: 0.80078125  Test accuracy: 0.8029
Epoch: 30  Loss: 0.54362077  Train accuracy: 0.8105469  Test accuracy: 0.8089
Epoch: 40  Loss: 0.44510463  Train accuracy: 0.8417969  Test accuracy: 0.8053
Epoch: 50  Loss: 0.5420026  Train accuracy: 0.78515625  Test accuracy: 0.7977
Epoch: 60  Loss: 0.49907446  Train accuracy: 0.8125  Test accuracy: 0.8135
Epoch: 70  Loss: 0.4249103  Train accuracy: 0.8378906  Test accuracy: 0.8199
Epoch: 80  Loss: 0.44562423  Train accuracy: 0.8261719  Test accuracy: 0.8245
Epoch: 90  Loss: 0.435998  Train accuracy: 0.83984375  Test accuracy: 0.8305
Epoch: 100  Loss: 0.4230092  Train accuracy: 0.84375  Test accuracy: 0.8258
